In [4]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm

In [2]:
# Full Core MIMIC-IV database path
core_mimiciv_path = '/export/scratch2/constellation-data/malafaia/physionet.org/files/mimiciv/1.0/'#'data/'
df_haim_ids = pd.read_csv(core_mimiciv_path + 'haim_mimiciv_key_ids.csv')
print(df_haim_ids.shape)

(34539, 3)


# check number of viable samples

In [5]:
to_remove = []
files = 0
total = 0
# remove example without images
for idx in range(df_haim_ids.shape[0]):
    filename = f"{idx:08d}" + '.pkl'
    filepath = core_mimiciv_path + 'pickle/' + filename
    with open(filepath, 'rb') as input:  
        patient = pickle.load(input)
    
    df_cxr = patient.cxr
    admittime = patient.admissions.admittime.values[0]
    dischtime = patient.admissions.dischtime.values[0]
    df_stay_cxr = df_cxr.loc[(df_cxr['charttime'] >= admittime) & (df_cxr['charttime'] <= dischtime)]
    if df_stay_cxr.shape[0] == 0: to_remove.append(filename)
    else: 
        files +=1
        total = total + df_stay_cxr['study_id'].unique().shape[0]

print(len(to_remove), " files without suitable cxr.")

15056  files without suitable cxr.


In [16]:
print(files)
print(total)

19483
104148


In [19]:
a = to_remove[-1]
print(a)

00034534.pkl


In [25]:
df_haim_ids.iloc[int(a[:-4])]

hadm_id       29653338.0
stay_id       38034263.0
subject_id    19295402.0
Name: 34534, dtype: float64

# read chartevents

In [27]:
df = pd.read_csv('/export/scratch2/constellation-data/malafaia/physionet.org/files/mimiciv/1.0/icu/chartevents.csv.gz')
df.head()

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10003700,28623837,30600691,2165-04-24 05:10:00,2165-04-24 05:11:00,228236,0.0,0.0,NaN,0
1,10003700,28623837,30600691,2165-04-24 05:12:00,2165-04-24 05:14:00,225067,0.0,0.0,NaN,0
2,10003700,28623837,30600691,2165-04-24 05:12:00,2165-04-24 05:14:00,225070,1.0,1.0,NaN,0
3,10003700,28623837,30600691,2165-04-24 05:12:00,2165-04-24 05:14:00,225076,1.0,1.0,NaN,0
4,10003700,28623837,30600691,2165-04-24 05:12:00,2165-04-24 05:14:00,225078,1.0,1.0,NaN,0


In [28]:
new_df = df[['subject_id', 'hadm_id', 'stay_id', 'charttime', 'storetime']]
new_df

,subject_id,hadm_id,stay_id,charttime,storetime
0,10003700,28623837,30600691,2165-04-24 05:10:00,2165-04-24 05:11:00
1,10003700,28623837,30600691,2165-04-24 05:12:00,2165-04-24 05:14:00
2,10003700,28623837,30600691,2165-04-24 05:12:00,2165-04-24 05:14:00
3,10003700,28623837,30600691,2165-04-24 05:12:00,2165-04-24 05:14:00
4,10003700,28623837,30600691,2165-04-24 05:12:00,2165-04-24 05:14:00
...,...,...,...,...,...
329499783,19999068,21606769,30143796,2161-08-30 20:00:00,2161-08-30 19:50:00
329499784,19999068,21606769,30143796,2161-08-30 20:00:00,2161-08-30 19:50:00
329499785,19999068,21606769,30143796,2161-08-30 20:00:00,2161-08-30 19:50:00
329499786,19999068,21606769,30143796,2161-08-30 21:00:00,2161-08-30 21:38:00


In [32]:
new_df['charttime'] = pd.to_datetime(new_df['charttime'])
new_df['storetime'] = pd.to_datetime(new_df['storetime'])
new_df = new_df.sort_values(by=['subject_id','hadm_id','stay_id'])

In [33]:
new_df.to_csv('/export/scratch2/constellation-data/malafaia/physionet.org/files/mimiciv/1.0/icu/short_chartevents.csv', index=None)

# re-process files
* remove unnecessary info
* redo df_haim_ids
* add extra column with number of samples per file

In [3]:
new_path = '/export/scratch2/constellation-data/malafaia/physionet.org/files/haim-mm-mafi/'

In [7]:
chartevents = pd.read_csv('/export/scratch2/constellation-data/malafaia/physionet.org/files/mimiciv/1.0/icu/short_chartevents.csv')
chartevents.head()

,subject_id,hadm_id,stay_id,charttime,storetime
0,10000032,29079034,39553978,2180-07-23 13:06:00,2180-07-23 13:07:00
1,10000032,29079034,39553978,2180-07-23 14:00:00,2180-07-23 14:20:00
2,10000032,29079034,39553978,2180-07-23 14:11:00,2180-07-23 14:17:00
3,10000032,29079034,39553978,2180-07-23 14:11:00,2180-07-23 14:17:00
4,10000032,29079034,39553978,2180-07-23 14:11:00,2180-07-23 14:17:00


In [10]:
new_ids = df_haim_ids
new_ids['pkl_id'] = range(new_ids.shape[0])
new_ids.head()

,hadm_id,stay_id,subject_id,pkl_id
0,29079034.0,39553978.0,10000032.0,0
1,26913865.0,39765666.0,10000980.0,1
2,24597018.0,37067082.0,10001217.0,2
3,27703517.0,34592300.0,10001217.0,3
4,26184834.0,37510196.0,10001884.0,4


In [6]:
# MIMICIV PATIENT CLASS STRUCTURE
class Patient_MM(object):
    def __init__(self, admissions, demographics, core,\
        labevents, prescriptions,\
        procedureevents, chartevents,\
        cxr, imcxr):
        
        ## CORE
        self.admissions = admissions
        self.demographics = demographics
        self.core = core
        ## HOSP
        self.labevents = labevents
        self.prescriptions = prescriptions
        ## ICU
        self.procedureevents = procedureevents
        self.chartevents = chartevents
        ## CXR
        self.cxr = cxr
        self.imcxr = imcxr

In [19]:
nfiles = len(new_ids)
no_samples = []
with tqdm(total = nfiles) as pbar:
    # Update process bar
    nbase= 0
    pbar.update(nbase)
    #iterate over all saved pkl files
    for i in range(new_ids.shape[0]):
        idx = int(new_ids['pkl_id'].iloc[i])
        filename = f"{idx:08d}" + '.pkl'
        # if file has valid samples
        if filename not in to_remove:
            #load file
            with open(core_mimiciv_path + 'pickle/' + filename, 'rb') as input:  
                old = pickle.load(input)
            
            # get number of samples
            df_cxr = old.cxr
            admittime = old.admissions.admittime.values[0]
            dischtime = old.admissions.dischtime.values[0]
            df_stay_cxr = df_cxr.loc[(df_cxr['charttime'] >= admittime) & (df_cxr['charttime'] <= dischtime)]
            no_samples.append(df_stay_cxr['study_id'].unique().shape[0])
                
            #get ids to select chartevent info
            info = new_ids.iloc[i]
            i_chartevents = chartevents[(chartevents.subject_id == info.subject_id) \
                & (chartevents.hadm_id == info.hadm_id) & (chartevents.stay_id == info.stay_id)]
            
            #define new patient
            new = Patient_MM(old.admissions, old.demographics, old.core,\
                            old.labevents, old.prescriptions,\
                            old.procedureevents, i_chartevents,\
                            old.cxr, old.imcxr)
            
            #save new file
            filepath = new_path + 'pickle/' + filename
            with open(filepath, 'wb') as output:
                pickle.dump(new, output, pickle.HIGHEST_PROTOCOL)
            
            print("Saved patient ", i)
        else: 
            print("Deleted patient ", i)
            new_ids.drop(i)

  0%|          | 0/34539 [00:00<?, ?it/s]

Deleted patient  0
Deleted patient  1
Deleted patient  2
Saved patient  3
Saved patient  4
Saved patient  5
Deleted patient  6
Saved patient  7
Saved patient  8
Saved patient  9
Deleted patient  10
Deleted patient  11
Deleted patient  12
Saved patient  13
Saved patient  14
Saved patient  15
Saved patient  16
Saved patient  17
Saved patient  18
Saved patient  19
Deleted patient  20
Deleted patient  21
Saved patient  22
Saved patient  23
Deleted patient  24
Saved patient  25
Saved patient  26
Deleted patient  27
Saved patient  28
Saved patient  29
Deleted patient  30
Deleted patient  31
Saved patient  32
Saved patient  33
Deleted patient  34
Saved patient  35
Saved patient  36
Deleted patient  37
Deleted patient  38
Saved patient  39
Saved patient  40
Saved patient  41
Saved patient  42
Saved patient  43
Saved patient  44
Deleted patient  45
Deleted patient  46
Deleted patient  47
Saved patient  48
Saved patient  49
Saved patient  50
Saved patient  51
Deleted patient  52
Saved patient  5

  0%|          | 0/34539 [14:50:32<?, ?it/s]

Saved patient  34538


# remove invalid files and add column with number of samples

In [14]:
all_ids = pd.read_csv(new_path + 'haim_ids_mm.csv')
all_ids.head()

,hadm_id,stay_id,subject_id,pkl_id,no_samples,split
0,27703517.0,34592300.0,10001217.0,3,1,train
1,26184834.0,37510196.0,10001884.0,4,5,train
2,23581541.0,39060235.0,10002013.0,5,3,train
3,23473524.0,35479615.0,10002428.0,7,9,train
4,28662225.0,33987268.0,10002428.0,8,13,train


In [15]:
# get list with valid pkl files
pkl_files = sorted(os.listdir(new_path + 'pickle/'))
a = pkl_files[1]
int(a[:-4])

4

In [16]:
nfiles = len(pkl_files)
pkl_ids = []
samples = []
studies = []
with tqdm(total = nfiles) as pbar:
    # Update process bar
    nbase= 0
    pbar.update(nbase)
    for pkl in pkl_files:
        pkl_id = int(pkl[:-4])
        pkl_ids.append(pkl_id)
        #load file
        with open(new_path + 'pickle/' + pkl, 'rb') as input:
            stay = pickle.load(input)
        # get number of samples
        df_cxr = stay.cxr
        admittime = stay.admissions.admittime.values[0]
        dischtime = stay.admissions.dischtime.values[0]
        df_stay_cxr = df_cxr.loc[(df_cxr['charttime'] >= admittime) & (df_cxr['charttime'] <= dischtime)]
        samples.append(df_stay_cxr.shape[0]) # following baseline
        studies.append(df_stay_cxr['study_id'].unique().shape[0])
        # Update process bar
        pbar.update(1)

  0%|          | 0/19483 [00:00<?, ?it/s]

100%|██████████| 19483/19483 [02:25<00:00, 133.84it/s]


In [19]:
# filter df according to valid pkl ids
new_ids = all_ids.loc[all_ids['pkl_id'].isin(pkl_ids)].copy()
new_ids['no_samples'] = samples
new_ids['no_studies'] = studies
print(new_ids.head())

      hadm_id     stay_id  subject_id  pkl_id  no_samples  split  no_studies
0  27703517.0  34592300.0  10001217.0       3           1  train           1
1  26184834.0  37510196.0  10001884.0       4           5  train           5
2  23581541.0  39060235.0  10002013.0       5           4  train           3
3  23473524.0  35479615.0  10002428.0       7           9  train           9
4  28662225.0  33987268.0  10002428.0       8          14  train          13


In [20]:
new_ids.head()
new_ids.to_csv(new_path + '/haim_ids_mm.csv', index=None)

# check train-test samples

In [94]:
split = []
for i in range(new_ids.shape[0]):
    idx = new_ids['pkl_id'].iloc[i]
    filename = f"{idx:08d}" + '.pkl'
    filepath = new_path + 'pickle/' + filename
    with open(filepath, 'rb') as input:  
        patient = pickle.load(input)
    split.append(patient.cxr.split[0])

In [95]:
new_ids['split'] = split
new_ids.to_csv(new_path + '/haim_ids_mm.csv', index=None)

In [101]:
new_ids[new_ids['split']=='train']

,hadm_id,stay_id,subject_id,pkl_id,no_samples,split
3,27703517.0,34592300.0,10001217.0,3,1,train
4,26184834.0,37510196.0,10001884.0,4,5,train
5,23581541.0,39060235.0,10002013.0,5,3,train
7,23473524.0,35479615.0,10002428.0,7,9,train
8,28662225.0,33987268.0,10002428.0,8,13,train
...,...,...,...,...,...,...
34532,22026077.0,33942755.0,19075794.0,34532,2,train
34535,26390110.0,36919705.0,19378228.0,34535,3,train
34536,25876676.0,39223303.0,19477304.0,34536,7,train
34537,24615413.0,38731785.0,19564054.0,34537,1,train


In [21]:
#new_ids['split'] = split
splits = ['train', 'validate', 'test']
for i in splits:
    a = new_ids[new_ids['split']==i]
    number_samples = a['no_samples'].sum()
    print(i, " -> ", number_samples)

train  ->  122255
validate  ->  1016
test  ->  2357


# access relevant data for one sample

In [1]:
new_path = '/export/scratch2/constellation-data/malafaia/physionet.org/files/haim-mm-mafi/'

In [2]:
# MIMICIV PATIENT CLASS STRUCTURE
class Patient_MM(object):
    def __init__(self, admissions, demographics, core,\
        labevents, prescriptions,\
        procedureevents, chartevents,\
        cxr, imcxr):
        
        ## CORE
        self.admissions = admissions
        self.demographics = demographics
        self.core = core
        ## HOSP
        self.labevents = labevents
        self.prescriptions = prescriptions
        ## ICU
        self.procedureevents = procedureevents
        self.chartevents = chartevents
        ## CXR
        self.cxr = cxr
        self.imcxr = imcxr

In [23]:
# get info from pickle file
# load pkl file
idx = 1155
filename = f"{idx:08d}" + '.pkl'
filepath = new_path + 'pickle/' + filename
with open(filepath, 'rb') as input:  
    stay = pickle.load(input)
# get tabular data
## admissions: ['ethnicity', 'marital_status', 'language', 'insurance']
## demographics: ['anchor_age', 'gender']
admissions = stay.admissions[['ethnicity', 'marital_status', 'language', 'insurance']].iloc[0]
demo = stay.demographics[['anchor_age', 'gender']].iloc[0]
tab = np.concatenate([admissions, demo])

# get cxr images
## get valid studies
df_meta = stay.cxr
admittime = stay.admissions.admittime.values[0]
dischtime = stay.admissions.dischtime.values[0]
df_meta = df_meta.loc[(df_meta['charttime'] >= admittime) & (df_meta['charttime'] <= dischtime)]
## get corresponding images
print("Idx for valid images", df_meta.index.tolist())
all_cxr = stay.imcxr
print("All images : ", len(all_cxr))
#valid_cxr = [stay.imcxr[i] for i in df_meta.index.tolist()]
#print("Valid images: ", len(valid_cxr), " out of ", len(all_cxr))

# get labels
df_meta = df_meta.loc[df_meta['Consolidation'].isin([0, 1])]
print("Idx for valid images", df_meta.index.tolist())
valid_cxr = [stay.imcxr[i] for i in df_meta.index.tolist()]
labels = df_meta['Consolidation']
print("Valid images: ", len(valid_cxr), " out of ", len(all_cxr))
print("Valid Labels: ", labels.tolist())

Idx for valid images [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 14, 16, 17]
All images :  18
Idx for valid images [1, 8, 14]
Valid images:  3  out of  18
Valid Labels:  [1.0, 1.0, 1.0]


In [63]:
df_meta

,dicom_id,study_id,subject_id,split,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,...,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,StudyDateForm,StudyTimeForm,cxrtime,charttime,deltacharttime
2,83234d34-0f431775-f0eaeeb6-c1c635b1-5de5dcfc,56420070.0,10430393.0,train,NaN,1.0,NaN,NaN,NaN,-1.0,...,21500121.0,54629.859,CHEST (PORTABLE AP),antero-posterior,Erect,2150-01-21,05:46:29.859000,2150-01-21 05:46:29.859,2150-01-21 05:46:00,23.733333
3,e7eeee32-5b202cc2-34a2bcd8-77184320-4501e5fd,52013278.0,10430393.0,train,NaN,NaN,NaN,NaN,NaN,NaN,...,21500123.0,83047.562,CHEST (PORTABLE AP),antero-posterior,Erect,2150-01-23,08:30:47.562000,2150-01-23 08:30:47.562,2150-01-23 08:30:00,74.466667
4,e2123012-8e1a0c8a-1c26577f-4ce36b19-67d8d9ed,53011104.0,10430393.0,train,NaN,NaN,NaN,NaN,NaN,NaN,...,21500120.0,134147.593,CHEST (PA AND LAT),lateral,Erect,2150-01-20,13:41:47.593000,2150-01-20 13:41:47.593,2150-01-20 13:41:00,7.650000
5,dd2beb81-30f5de84-99e0c802-6ce86d01-a259cec5,53011104.0,10430393.0,train,NaN,NaN,NaN,NaN,NaN,NaN,...,21500120.0,134147.593,CHEST (PA AND LAT),antero-posterior,Erect,2150-01-20,13:41:47.593000,2150-01-20 13:41:47.593,2150-01-20 13:41:00,7.650000
6,6daa46f9-bf7be733-9ec19664-679d913d-c0e085c6,55961303.0,10430393.0,train,NaN,1.0,NaN,NaN,NaN,NaN,...,21500122.0,53334.406,CHEST (PORTABLE AP),antero-posterior,Erect,2150-01-22,05:33:34.406000,2150-01-22 05:33:34.406,2150-01-22 05:33:00,47.516667


# check number of samples per label

In [66]:
new_path = '/export/scratch2/constellation-data/malafaia/physionet.org/files/haim-mm-mafi/'
ids_path = new_path + 'haim_ids_mm.csv'
new_ids = pd.read_csv(ids_path)

In [67]:
labels = ['Atelectasis', 'Cardiomegaly',
       'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
       'Lung Lesion', 'Lung Opacity',
       'Pneumonia', 'Pneumothorax']

In [76]:
nfiles = len(new_ids)
stay_counts = np.zeros([10,1])
sample_counts = np.zeros([10,1])

with tqdm(total = nfiles) as pbar:
    # Update process bar
    nbase= 0
    pbar.update(nbase)
    #iterate over all saved pkl files
    for i in range(new_ids.shape[0]):
        idx = int(new_ids['pkl_id'].iloc[i])
        filename = f"{idx:08d}" + '.pkl'
        #load file
        with open(new_path + 'pickle/' + filename, 'rb') as input:  
            stay = pickle.load(input)    
        
        # get labels
        df_meta = stay.cxr
        admittime = stay.admissions.admittime.values[0]
        dischtime = stay.admissions.dischtime.values[0]
        df_meta = df_meta.loc[(df_meta['charttime'] >= admittime) & (df_meta['charttime'] <= dischtime)]
        for j in range(len(labels)):
            label = labels[j]
            temp = df_meta.loc[df_meta[label].isin([0, 1])]
            #print(temp)
            sample_counts[j] = sample_counts[j] + temp.shape[0]
            if temp.shape[0] > 0:
                stay_counts[j] += 1
                
        pbar.update(1)

100%|██████████| 19483/19483 [03:05<00:00, 105.07it/s]


In [77]:
for i in range(len(labels)):
    print(labels[i], " -> ", stay_counts[i], " valid files and ", sample_counts[i], " samples")

Atelectasis  ->  [11658.]  valid files and  [42147.]  samples
Cardiomegaly  ->  [13961.]  valid files and  [48493.]  samples
Consolidation  ->  [5862.]  valid files and  [13236.]  samples
Edema  ->  [12774.]  valid files and  [42464.]  samples
Enlarged Cardiomediastinum  ->  [4886.]  valid files and  [9059.]  samples
Fracture  ->  [970.]  valid files and  [2298.]  samples
Lung Lesion  ->  [1203.]  valid files and  [2368.]  samples
Lung Opacity  ->  [11516.]  valid files and  [40695.]  samples
Pneumonia  ->  [9103.]  valid files and  [19934.]  samples
Pneumothorax  ->  [12721.]  valid files and  [47079.]  samples


# build tab features csv

In [115]:
nfiles = len(new_ids)
# get tabular data
## admissions: ['ethnicity', 'marital_status', 'language', 'insurance']
## demographics: ['anchor_age', 'gender']
cols = ['anchor_age', 'gender', 'ethnicity', 'marital_status', 'language', 'insurance', 'pkl_id']
df_tab = pd.DataFrame(columns=cols)

with tqdm(total = nfiles) as pbar:
    # Update process bar
    nbase= 0
    pbar.update(nbase)
    #iterate over all saved pkl files
    for i in range(new_ids.shape[0]):
        idx = int(new_ids['pkl_id'].iloc[i])
        filename = f"{idx:08d}" + '.pkl'
        #load file
        with open(new_path + 'pickle/' + filename, 'rb') as input:  
            stay = pickle.load(input) 
        admissions = stay.admissions[['ethnicity', 'marital_status', 'language', 'insurance']].iloc[0]
        demo = stay.demographics[['anchor_age', 'gender']].iloc[0]
        row_df = pd.concat([demo, admissions])
        row_df = pd.DataFrame([row_df])
        row_df['pkl_id'] = idx
        df_tab = pd.concat([df_tab, row_df], ignore_index=True)
        pbar.update(1)
        

  0%|          | 10/19483 [00:00<03:31, 92.12it/s]

100%|██████████| 19483/19483 [03:35<00:00, 90.21it/s] 


In [126]:
cat_fts = ['gender', 'ethnicity', 'marital_status', 'language', 'insurance']
for ft in cat_fts:
    print(ft, df_tab[ft].unique())
    

gender ['F' 'M']
ethnicity ['WHITE' 'BLACK/AFRICAN AMERICAN' 'OTHER' 'UNABLE TO OBTAIN' 'UNKNOWN'
 'HISPANIC/LATINO' 'ASIAN' 'AMERICAN INDIAN/ALASKA NATIVE']
marital_status ['MARRIED' 'SINGLE' 'WIDOWED' 'DIVORCED' nan]
language ['?' 'ENGLISH']
insurance ['Other' 'Medicare' 'Medicaid']


In [135]:
# process tab_fts
df_tab_new = pd.get_dummies(df_tab, columns=cat_fts, drop_first=False)
df_tab_new = df_tab_new.drop(columns=['gender_M', 'ethnicity_UNABLE TO OBTAIN', 'ethnicity_UNKNOWN', 'language_?', 'insurance_Other'])
df_tab_new

,anchor_age,pkl_id,gender_F,ethnicity_AMERICAN INDIAN/ALASKA NATIVE,ethnicity_ASIAN,ethnicity_BLACK/AFRICAN AMERICAN,ethnicity_HISPANIC/LATINO,ethnicity_OTHER,ethnicity_WHITE,marital_status_DIVORCED,marital_status_MARRIED,marital_status_SINGLE,marital_status_WIDOWED,language_ENGLISH,insurance_Medicaid,insurance_Medicare
0,55.0,3,1,0,0,0,0,0,1,0,1,0,0,0,0,0
1,68.0,4,1,0,0,1,0,0,0,0,1,0,0,1,0,1
2,53.0,5,1,0,0,0,0,1,0,0,0,1,0,1,0,1
3,80.0,7,1,0,0,0,0,0,1,0,0,0,1,1,0,1
4,80.0,8,1,0,0,0,0,0,1,0,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19478,56.0,34532,0,0,0,0,0,0,0,0,1,0,0,1,0,0
19479,41.0,34535,0,0,0,0,0,0,1,0,0,1,0,1,0,0
19480,80.0,34536,0,0,0,0,0,0,1,0,0,0,1,0,1,0
19481,73.0,34537,0,0,0,0,0,0,1,0,0,1,0,1,0,1


In [136]:
df_tab_new.to_csv(new_path + 'tabular_mm.csv', index=None)

# adpat csv (ids) for consolidation prediction

In [19]:
new_path = '/export/scratch2/constellation-data/malafaia/physionet.org/files/haim-mm-mafi/'
new_ids = pd.read_csv(new_path + 'haim_ids_mm.csv')
new_ids.head()

,hadm_id,stay_id,subject_id,pkl_id,no_samples,split,no_studies
0,27703517.0,34592300.0,10001217.0,3,1,train,1
1,26184834.0,37510196.0,10001884.0,4,5,train,5
2,23581541.0,39060235.0,10002013.0,5,4,train,3
3,23473524.0,35479615.0,10002428.0,7,9,train,9
4,28662225.0,33987268.0,10002428.0,8,14,train,13


In [8]:
# MIMICIV PATIENT CLASS STRUCTURE
class Patient_MM(object):
    def __init__(self, admissions, demographics, core,\
        labevents, prescriptions,\
        procedureevents, chartevents,\
        cxr, imcxr):
        
        ## CORE
        self.admissions = admissions
        self.demographics = demographics
        self.core = core
        ## HOSP
        self.labevents = labevents
        self.prescriptions = prescriptions
        ## ICU
        self.procedureevents = procedureevents
        self.chartevents = chartevents
        ## CXR
        self.cxr = cxr
        self.imcxr = imcxr

In [31]:
# remove pkl files without consolidation valid samples
label = 'Consolidation'
nfiles = len(new_ids)
to_keep = []
y_counts = []

with tqdm(total = nfiles) as pbar:
    # Update process bar
    nbase= 0
    pbar.update(nbase)
    #iterate over all saved pkl files
    for i in range(new_ids.shape[0]):
        idx = int(new_ids['pkl_id'].iloc[i])
        filename = f"{idx:08d}" + '.pkl'
        #load file
        with open(new_path + 'pickle/' + filename, 'rb') as input:  
            stay = pickle.load(input)  
        # get labels
        df_meta = stay.cxr
        admittime = stay.admissions.admittime.values[0]
        dischtime = stay.admissions.dischtime.values[0]
        df_meta = df_meta.loc[(df_meta['charttime'] >= admittime) & (df_meta['charttime'] <= dischtime)]
        # select samples with 0 or 1 for the (consolidation) class
        temp = df_meta.loc[df_meta[label].isin([0, 1])]
        if temp.shape[0] > 0: # there are valid samples (1 or 0)
            to_keep.append(idx) # keep patient for label (consolidation) prediction
            y_counts.append(temp[label].sum()) # save number of positive samples
                
        pbar.update(1)

  0%|          | 0/19483 [00:00<?, ?it/s]

(224, 224)


In [29]:
consolidation_ids = new_ids.loc[new_ids['pkl_id'].isin(to_keep)].copy()
consolidation_ids['y_counts'] = y_counts
consolidation_ids['y_counts'] = consolidation_ids['y_counts'].astype(int)
consolidation_ids['y_ratio'] = (consolidation_ids['y_counts']/consolidation_ids['no_samples'])#>0.5).astype(int)
consolidation_ids

,hadm_id,stay_id,subject_id,pkl_id,no_samples,split,no_studies,y_counts,y_ratio
6,22774359.0,30676350.0,10003019.0,13,1,train,1,1,1.000000
7,20214994.0,32128372.0,10003400.0,14,9,train,7,2,0.222222
15,20338077.0,35128235.0,10007928.0,25,3,train,3,0,0.000000
16,22995465.0,35636875.0,10009049.0,26,2,train,2,1,0.500000
17,22429197.0,39880770.0,10010867.0,28,15,train,15,5,0.333333
...,...,...,...,...,...,...,...,...,...
19462,27218949.0,39024906.0,14371035.0,34494,11,train,11,4,0.363636
19475,21957886.0,39330295.0,18137539.0,34522,18,train,13,3,0.166667
19479,26390110.0,36919705.0,19378228.0,34535,3,train,3,0,0.000000
19480,25876676.0,39223303.0,19477304.0,34536,8,train,7,1,0.125000


In [30]:
consolidation_ids.to_csv(new_path + 'consolidation_ids.csv', index=None)

# split data

In [36]:
# split data in 5 groups with same class imbalance
info = consolidation_ids.copy()
info['label'] = (info['y_ratio']>0.5).astype(int)
print(info['label'].sum()/info.shape[0])

0.09672466734902764


In [38]:
from sklearn.model_selection import train_test_split

In [42]:
pkl_id = np.asarray(info['pkl_id'])
y = np.asarray(info['label'])
id_main, id_test, label_main, label_test = train_test_split(pkl_id, y, test_size = 0.20, random_state=0, stratify = y)
X_test = info[info['pkl_id'].isin(id_test)]
y_test = X_test['label']
X_main = info[info['pkl_id'].isin(id_main)]
y_main = X_main['label']

In [45]:
X_test['label'].sum()/X_test.shape[0]

0.09633418584825235

# other

In [4]:
# Supply the haim id patient you would like to process
haim_patient_idx = 35
#Load precomputed file
filename = f"{haim_patient_idx:08d}" + '.pkl'
with open(core_mimiciv_path + 'pickle/' + filename, 'rb') as input:  
        patient = pickle.load(input)

In [8]:
# Get information of chest x-rays conducted within this patient stay
df_cxr = patient.cxr
df_imcxr = patient.imcxr
admittime = patient.admissions.admittime.values[0]
dischtime = patient.admissions.dischtime.values[0]
df_stay_cxr = df_cxr.loc[(df_cxr['charttime'] >= admittime) & (df_cxr['charttime'] <= dischtime)]
print("admittime: ", admittime)
print("dischtime: ", dischtime)
print(df_stay_cxr.shape[0], "image instances")

admittime:  2175-09-06T19:46:00.000000000
dischtime:  2175-09-14T19:23:00.000000000
2 image instances


In [ ]:
if not df_stay_cxr.empty:
    print("df_stay_cxr not empty!")
    # iterate over each cxr instance
    for idx, df_stay_cxr_row in df_stay_cxr.iterrows():
        # Get stay anchor times
        img_charttime = df_stay_cxr_row['charttime']
        img_deltacharttime = df_stay_cxr_row['deltacharttime']
        print('cxr charttime: ', img_charttime)
        print('cxr deltacharttime: ', img_deltacharttime)

        # Get time to discharge and discharge location/status
        img_id = df_stay_cxr_row["dicom_id"]
        print("img_id: ", img_id)
        img_length_of_stay = delta_t = (dischtime-img_charttime).total_seconds()/3600 # Result in hrs
        discharge_location = patient.core['discharge_location'][0]
        if discharge_location == "DIED": death_status = 1
        else: death_status = 0
            
        # Select allowed timestamp range
        start_hr = None
        end_hr = img_deltacharttime
        
        dt_patient = get_timebound_patient_icustay(patient, start_hr , end_hr)
        print("patient icustay: ", dt_patient)
        is_included = True

        if is_included:
            print("patient is included!")
            df_init = pd.DataFrame([[img_id, img_charttime, img_deltacharttime, discharge_location, img_length_of_stay, death_status]],columns=['img_id', 'img_charttime', 'img_deltacharttime', 'discharge_location', 'img_length_of_stay', 'death_status'])
            print("starting embeddings...")
            df_fusion = process_cxr_embeddings_haim_id(haim_id, dt_patient, df_init)
            print("finished embeddings!")
            
            if os.path.isfile(fname):
                df_fusion.to_csv(fname, mode='a', index=False, header=False)
            else:
                df_fusion.to_csv(fname, mode='w', index=False)
                